In [1]:
#%pylab inline

import h5py
from multiprocessing import Pool
import time
import warnings

import numpy as np
import scipy

import datetime as d
from collections import Counter

import obspy

import pandas as pd
import itertools



np.random.seed(7)

import sys

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
event = {
    'name'       : 'Japan',
    'center'     : [38.0, 140.0],
    'delta'      : 15.0, 
    'min_mag'    : 0.0,
    'start_date' : '1964-01-01', 
    'end_date'   : '2019-01-01',
    'client'     : 'ISC'
}

figname = '{}_{}_'.format(event['name'], event['client'])

In [3]:
catalog = pd.read_pickle('Japan_ISC_Complete_NaN.pkl')

catalog_np = np.array(catalog.iloc[:,4:])
catalog_mags = np.nanmax(catalog_np, axis=1)
catalog_dates = np.array(catalog.iloc[:, 0])
catalog_coords = np.array(catalog.iloc[:,1:3])
catalog_depth = np.array(catalog.iloc[:, 3])

In [5]:
def Ms2Mw(Ms):
    return np.where(
        Ms == Ms, 
        np.where(Ms < 6.15, 0.65 * Ms + 2.2, Ms - 0.02), # 3.0 - 6.1 | 6.2 - 8.0
        np.nan) 
    
def mb2Mw(mb):
    return 0.85*mb + 1.02
    
def M2Mw(M):
    return np.where(
        M == M, 
        np.where(M < 5.55, 0.58*M+2.25, 0.97*M+0.04), # 3-5.5 | 5.6 - 8.2
        np.nan) 


def fillWith(Mw, Mn):
    return np.where(Mn==Mn, Mn, Mw)

def doMwConversion():
    Mw = M2Mw(np.array(catalog_np[:, 39]))

    Mw = fillWith(Mw, M2Mw(np.array(catalog_np[:, 42])))
    Mw = fillWith(Mw, M2Mw(np.array(catalog_np[:, 35])))
    Mw = fillWith(Mw, mb2Mw(np.array(catalog_np[:, 0])))
    Mw = fillWith(Mw, Ms2Mw(np.array(catalog_np[:, 5])))

    return Mw 


def doMwConversionMix():
    Mw = M2Mw(np.array(catalog_np[:, 39]))

    Mw = fillWith(Mw, np.array(catalog_np[:, 42]))
    Mw = fillWith(Mw, np.array(catalog_np[:, 35]))
    Mw = fillWith(Mw, np.array(catalog_np[:, 0]))
    Mw = fillWith(Mw, np.array(catalog_np[:, 5]))

    return Mw 

In [5]:
catalog_mags = doMwConversion()

catalog_mags_mix = doMwConversionMix()

In [7]:
def calcB(bmagsin, bmin=3):  
    bmags = bmagsin[~np.isnan(bmagsin)]
    bmags = bmags[bmags>=bmin]
    if len(bmags) < 2: 
        return np.nan
    b1 = np.sum(bmags)/len(bmags) - np.min(bmags)
    if b1 == 0.0: 
        return np.nan 
    return 1.0/b1*np.log10(np.e), 1.96/np.sqrt(len(bmags))/b1*np.log10(np.e) # Last part is the uncertainty


## Earthquake Animation, red x black bock starts here

In [ ]:


# Depth in km:
depth0 = 0
depth1 = 70

# Relations from NARW-2005.pdf
# mb: 0
# MS: 5
# ML: 35
# M:  39
# MV: 42

def Ms2Mw(Ms):
    return np.where(
        Ms == Ms, 
        np.where(Ms < 6.15, 0.65 * Ms + 2.2, Ms - 0.02), # 3.0 - 6.1 | 6.2 - 8.0
        np.nan) 
    
def mb2Mw(mb):
    return 0.85*mb + 1.02
    
def M2Mw(M):
    return np.where(
        M == M, 
        np.where(M < 5.55, 0.58*M+2.25, 0.97*M+0.04), # 3-5.5 | 5.6 - 8.2
        np.nan) 


def fillWith(Mw, Mn):
    return np.where(Mn==Mn, Mn, Mw)
    
Mw = M2Mw(np.array(catalog_np[:, 39]))

Mw = fillWith(Mw, M2Mw(np.array(catalog_np[:, 42])))
Mw = fillWith(Mw, M2Mw(np.array(catalog_np[:, 35])))
Mw = fillWith(Mw, mb2Mw(np.array(catalog_np[:, 0])))
Mw = fillWith(Mw, Ms2Mw(np.array(catalog_np[:, 5])))



catalog_mags = Mw # Nans are skipped with >= step 


# Create Block

In [7]:
depth0 = -1
depth1 = 70

In [11]:
def processDayBlock(e):
    long_min = 120.0
    long_max = 150.0
    lat_min  = 20.0
    lat_max  = 50.0
    gridsize = 0.1
    dr = 0.25 # Radius for b calculation
    
    def calcB(bmags):  
        if len(bmags) < 2: 
            return np.nan
        b1 = np.sum(bmags)/len(bmags) - np.min(bmags)
        if b1 == 0.0: 
            return np.nan 
        return 1.0/b1*np.log10(np.e) # 1.96/np.sqrt(len(bmags))/b1*np.log10(np.e) # Last part is the uncertainty
    
    # make selection
    sel = np.logical_and(
            np.logical_and(
                catalog_dates > e - np.timedelta64(513, 'D'),
                catalog_dates < e - np.timedelta64(1, 'D')),
            np.logical_and(catalog_depth/1000 <= depth1, 
                           catalog_depth/1000 > depth0))
    sel = np.logical_and(sel, catalog_mags == catalog_mags) # Filter out nan Magnitudes
    s_long = catalog_coords[sel, 1]
    s_lats = catalog_coords[sel, 0]
    s_mags = catalog_mags[sel]
    s_date = catalog_dates[sel]
    
    #calculate points:
    px, py = np.meshgrid(np.arange(long_min+0.05, long_max, 0.1), np.arange( lat_min+0.05,  lat_max, 0.1))
    points = np.reshape(np.append(px, py), (-1,2), order='F')
    sel_points = np.array([((p[0]-s_long)**2 + (p[1]-s_lats)**2) < dr for p in points])
  
    b_values = np.zeros((300*300), dtype='float16')
    n_values = np.zeros((300*300), dtype='int')
    for k, sel_point in enumerate(sel_points):
        k_mags = s_mags[sel_point]
        b_values[k] = calcB(k_mags)
        n_values[k] = len(k_mags)
        
    print('Done With {}!'.format(e), end='\r')
    return np.reshape(b_values, (300,300)), np.reshape(n_values, (300,300))

In [8]:
def processDayMetadata(e):
    long_min = 120.0
    long_max = 150.0
    lat_min  = 20.0
    lat_max  = 50.0
    gridsize = 0.1
    dr = 0.25 # Radius for b calculation
    
    # make selection
    sel = np.logical_and(
            np.logical_and(
                catalog_dates > e - np.timedelta64(1, 'D'),
                catalog_dates < e),
            np.logical_and(catalog_depth/1000 <= depth1, 
                           catalog_depth/1000 > depth0))
    sel = np.logical_and(sel, catalog_mags == catalog_mags) # Filter out nan Magnitudes
    s_long = catalog_coords[sel, 1]
    s_lats = catalog_coords[sel, 0]
    s_mags = catalog_mags[sel]
    s_depth = catalog_depth[sel]  ### JKC: sel was missing here!?!?!?!?!?!?!

    gix = lambda a : min(int((a - long_min) / gridsize), 299)
    giy = lambda a : min(int((a - lat_min) / gridsize), 299)
    
    depth = np.ones((300,300), dtype='float32')*np.nan
    maxmag = np.ones((300,300), dtype='float16') * -1.0

    for i in range(len(s_long)):
        if maxmag[gix(s_long[i]), giy(s_lats[i])] < s_mags[i]:
            maxmag[gix(s_long[i]), giy(s_lats[i])] = s_mags[i] 
            depth[gix(s_long[i]), giy(s_lats[i])] = s_depth[i]
    
    
    #print('Done With {}!'.format(e), end='\r')
    return maxmag, depth

In [10]:
# Code for full regeneration of file

fname = 'ML_Tiles_000to070_FullBlock_sliced.hdf5'
dates = np.arange('2000-01-01', '2020-01-01', dtype='datetime64[D]') # Full block starts at 2000-01-01, what was i thinking

t1 = time.time()
with Pool(20) as p: # do not use more than 60 or RAM will run out!
    res = p.map(processDayBlock, dates)
    f1 = h5py.File(fname, 'a')
    for i, r in enumerate(res):
        g1 = f1.create_group('day_{:04d}_{}'.format(i, str(dates[i])))
        g1.create_dataset('b_value', r[0].shape, dtype='float16', data=r[0])
        g1.create_dataset('n_eq', r[1].shape, dtype='int16', data=r[1])
    f1.close()
    t2 = time.time()
    res = p.map(processDayMetadata, dates)
    f1 = h5py.File(fname, 'a')
    for i, r in enumerate(res):
        g1 = f1['day_{:04d}_{}'.format(i, str(dates[i]))]
        g1.create_dataset('maxmag', r[0].shape, dtype='float16', data=r[0])
        g1.create_dataset('depth', r[1].shape, dtype='float32', data=r[1])
    f1.close()
    t2 = time.time()


print('Calculation: {} s'.format(t2-t1))

NameError: name 'processDayBlock' is not defined

In [9]:
# Update faulty depth

fname = 'ML_Tiles_000to070_FullBlock_sliced.hdf5'
dates = np.arange('2000-01-01', '2020-01-01', dtype='datetime64[D]') # Full block starts at 2000-01-01, what was i thinking

t1 = time.time()
with Pool(20) as p: # do not use more than 60 or RAM will run out!
    res = p.map(processDayMetadata, dates)
    f1 = h5py.File(fname, 'a')
    for i, r in enumerate(res):
        g1 = f1['day_{:04d}_{}'.format(i, str(dates[i]))]
        g1['maxmag'][...] = r[0]
        g1['depth'][...] = r[1]
        #g1.create_dataset('maxmag', r[0].shape, dtype='float16', data=r[0])
        #g1.create_dataset('depth', r[1].shape, dtype='float32', data=r[1])
    f1.close()
    t2 = time.time()


print('Calculation: {} s'.format(t2-t1))

Calculation: 260.14920377731323 s


In [10]:
f2 = h5py.File('ML_Tiles_000to070_FullBlock.hdf5', 'w')
f3 = h5py.File('ML_Tiles_000to070_FullBlock_sliced.hdf5', 'r')

dates = np.arange('2000-01-01', '2020-01-01', dtype='datetime64[D]')

keys = f3.keys()
keys = sorted(np.array([str(key) for key in keys]))

fullBlock_b = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_n = np.zeros((len(keys), 300, 300), dtype=int)
fullBlock_maxmag = np.zeros((len(keys), 300, 300), dtype='float16')
fullBlock_depth = np.zeros((len(keys), 300, 300), dtype='float32')

for i, key in enumerate(keys):
    fullBlock_b[i] = np.array(f3[key+'/b_value'])
    fullBlock_n[i] = np.array(f3[key+'/n_eq'])
    fullBlock_maxmag[i] = np.array(f3[key+'/maxmag']).T
    fullBlock_depth[i]  = np.array(f3[key+'/depth']).T
f3.close()



f2.create_dataset('b_value', fullBlock_b.shape, dtype='float16', data=fullBlock_b)
f2.create_dataset('n_eq', fullBlock_n.shape, dtype='int16', data=fullBlock_n)
f2.create_dataset('maxmag', fullBlock_b.shape, dtype='float16', data=fullBlock_maxmag)
f2.create_dataset('depth', fullBlock_n.shape, dtype='float32', data=fullBlock_depth)
f2.close()


In [3]:
f2 = h5py.File('ML_Tiles_000to070_FullBlock.hdf5', 'r')
bval_loc = np.array(f2['b_value'])
n_eq_avg = np.array(f2['n_eq'])
maxm_loc = np.array(f2['maxmag'])
dept_avg = np.array(f2['depth'])
f2.close()


def makeNoEQArray():
    noeqarray = np.zeros(maxm_loc.shape-np.array([512, 32,32]), dtype=bool)
    for i in range(512, maxm_loc.shape[0]):
        for j in range(16, maxm_loc.shape[1]-16):
            for k in range(16, maxm_loc.shape[2]-16):
                if np.max(maxm_loc[i-7:i+7, j-8:j+8, k-8:k+8]) < 4.5:
                    if np.mean(n_eq_avg[i-7:i+7, j-8:j+8, k-8:k+8]) > 10:
                        noeqarray[i-512, j-16, k-16] = True
        print(i, end='\r')
    return noeqarray

noeqarray = makeNoEQArray()

f2 = h5py.File('ML_Tiles_000to070_FullBlock.hdf5', 'a')
f2.create_dataset('NoEQArray', noeqarray.shape, dtype='float16', data=noeqarray)
f2.close()

KeyboardInterrupt: 

### Copy noeqarray over

In [6]:
fold = h5py.File('ML_Tiles_000to070_FullBlock_old.hdf5', 'r')
noeqarray = np.array(fold['NoEQArray'])
fold.close()

print('Read old data')

f2 = h5py.File('ML_Tiles_000to070_FullBlock.hdf5', 'a')
f2.create_dataset('NoEQArray', noeqarray.shape, dtype='float16', data=noeqarray)
f2.close()

Read old data


# End Create Block!

In [ ]:
f2 = h5py.File('ML_Tiles_000to070_FullBlock.hdf5', 'r')

bval_avg = np.mean(f2['b_value'], axis=0)
n_eq_avg = np.mean(f2['n_eq'], axis=0)
maxm_avg = np.mean(f2['maxmag'], axis=0)
dept_avg = np.nanmean(f2['depth'], axis=0)

f2.close()